In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import ast
import cv2

from single_dataset import DATA_PATH
from single_train import N_FOLDS, SEED

In [2]:
sagittal_t2 = pd.read_csv("../rsna24-data/saggittal_t2.csv")
sagittal_t1 = pd.read_csv("../rsna24-data/sagittal_t1.csv")
axial_t2 = pd.read_csv("../rsna24-data/axial_t2.csv")

In [3]:
def apply(x):
    x = x.sort_values("instance_number")
    x = [ast.literal_eval(x) for x in x.preds.values]
    return cv2.resize(np.array(x), (240, 240), interpolation=cv2.INTER_CUBIC)

In [4]:
sagittal_t2 = sagittal_t2.groupby(["study_id", "series_id", "series_description"]).apply(apply)
sagittal_t1 = sagittal_t1.groupby(["study_id", "series_id", "series_description"]).apply(apply)
axial_t2 = axial_t2.groupby(["study_id", "series_id", "series_description"]).apply(apply)
df = pd.concat([sagittal_t1, axial_t2, sagittal_t2])
del sagittal_t1, axial_t2, sagittal_t2

In [5]:
df.head()

study_id  series_id   series_description
4003253   1054713880  Sagittal T1           [[[0.9425638319896079, 0.045684960292843635, 0...
4646740   3486248476  Sagittal T1           [[[0.7141367226191223, 0.2491852240339252, 0.0...
7143189   3219733239  Sagittal T1           [[[0.9668390327396587, 0.028452445823666253, 0...
8785691   1570286759  Sagittal T1           [[[0.9496683195680026, 0.039946704301318345, 0...
10728036  2399638375  Sagittal T1           [[[0.888722207309249, 0.11348500866120123, -0....
dtype: object

In [6]:
df = df.reset_index()
df.head()

,study_id,series_id,series_description,0
0,4003253,1054713880,Sagittal T1,"[[[0.9425638319896079, 0.045684960292843635, 0..."
1,4646740,3486248476,Sagittal T1,"[[[0.7141367226191223, 0.2491852240339252, 0.0..."
2,7143189,3219733239,Sagittal T1,"[[[0.9668390327396587, 0.028452445823666253, 0..."
3,8785691,1570286759,Sagittal T1,"[[[0.9496683195680026, 0.039946704301318345, 0..."
4,10728036,2399638375,Sagittal T1,"[[[0.888722207309249, 0.11348500866120123, -0...."


In [22]:
df = df.groupby(["study_id"]).apply(lambda x: np.vstack([np.array(y).transpose((2, 0, 1)) for y in x.sort_values(by="series_description", ascending=False)[0].values]).shape)
df.head()

study_id
4003253     (9, 240, 240)
4646740     (9, 240, 240)
7143189     (9, 240, 240)
8785691     (9, 240, 240)
10728036    (9, 240, 240)
dtype: object

In [23]:
df.columns = ["study_id", "image"]

In [27]:
df.head()

study_id
4003253     (9, 240, 240)
4646740     (9, 240, 240)
7143189     (9, 240, 240)
8785691     (9, 240, 240)
10728036    (9, 240, 240)
dtype: object